In [3]:
import numpy as np
import math
import random

In [4]:
DISCOUNT_RATE = 0.9
LOC_1_EXPECTED_DEMAND = 3
LOC_1_EXPECTED_RETURNS = 3
LOC_2_EXPECTED_DEMAND = 4
LOC_2_EXPECTED_RETURNS = 2
MAX_MOVE = 5
MAX_CAPACITY = 20
RENTAL_REWARD = 10
MOVE_REWARD = -2

In [5]:
# State is the number of cars at each location at the end of each day
# It doesn't matter whether they've just been returned, or weren't rented,
# as both types of vehicle won't be rented today (rentals are over) and
# could be rented out tomorrow
def state_iter():
    for loc_1_count in range(MAX_CAPACITY+1):
        for loc_2_count in range(MAX_CAPACITY+1):
            yield (loc_1_count,loc_2_count)

def action_iter():
    for transfer in range(-MAX_MOVE,MAX_MOVE+1):
        yield transfer

In [6]:
def poisson_prob(n,la):
    return math.exp(-la) * la**n / math.factorial(n)

# exclude anything with less than 1% chance of occurrence, should speed up but not impact outcome much/at all for this prob.
def poisson_probs(la):
    return [poisson_prob(x, la) for x in range(0,MAX_CAPACITY+1) if poisson_prob(x,la) > 0.01]

In [7]:
def expected_return(state, action, values):
    # Action is given and has known cost
    Vsa = 0
    # Check if the action is 'legal', if not return -inf
    loc_1_avail, loc_2_avail = state
    if action > loc_1_avail or action < -loc_2_avail:
        return -float('inf')
    # Any move has a known cost with prob 1
    Vsa += MOVE_REWARD * abs(action)
    # Update availability based on action; enforce max capacity constraint
    loc_1_avail = min(loc_1_avail - action, MAX_CAPACITY)
    loc_2_avail = min(loc_2_avail + action, MAX_CAPACITY)
    
    for loc_1_dem, loc_1_dem_prob in enumerate(loc_1_dem_probs):
        for loc_2_dem, loc_2_dem_prob in enumerate(loc_2_dem_probs):
            for loc_1_ret, loc_1_ret_prob in enumerate(loc_1_ret_probs):
                for loc_2_ret, loc_2_ret_prob in enumerate(loc_2_ret_probs):
                    # Reward is fulfilled demand * RENTAL_REWARD
                    prob = loc_1_dem_prob * loc_2_dem_prob * loc_1_ret_prob * loc_2_ret_prob
                    loc_1_rented = min(loc_1_avail, loc_1_dem)
                    loc_2_rented = min(loc_2_avail, loc_2_dem)
                    rental_reward = (loc_1_rented + loc_2_rented) * RENTAL_REWARD
                    loc_1_avail_final = min(loc_1_avail - loc_1_rented + loc_1_ret, MAX_CAPACITY)
                    loc_2_avail_final = min(loc_2_avail - loc_2_rented + loc_2_ret, MAX_CAPACITY)
                    next_state_value = values[(loc_1_avail_final, loc_2_avail_final)]
                    Vsa += prob * (rental_reward + next_state_value * DISCOUNT_RATE)
    return Vsa
    
    
    

In [8]:
def greedyAction(state, values):
    # Generation action/return pairs for all actions
    options = [(action, expected_return(state, action, values)) for action in action_iter()]
    # Return the action with the max expected return
    return max(options, key=lambda x: x[1])[0]

In [9]:
def print_policy(policy):
    print_rows = ['']
    row_count = 0
    for key, value in policy.items():
        if key[0] > row_count:
            row_count += 1
            print_rows.append('')
        print_rows[row_count] += str(abs(value))
    print('\n'.join(print_rows))

In [10]:
# Desired output is optimal policy
# Initialize Val and Pol for all s
# s is all possible end-of-day states of vehicle count
# anything from 0 to max_capacity for each branch

# The action for a state is a transfer of vehicles, which results in a new state
# From the new state there's rentals and returns.
# Rentals result in revenue (up to a limit of state availability)
# Returns - Rentals give the end state.
# That end state will have new actions and new probabilities and new states
# ad infinitum... but we don't have to follow all of those, because they're
# encapsulated by the value associated with the new state, which we can just look up.
# The value of the current state is based on the probability weighted values of possible future states
# appropriately discounted.

# Precalc and cache poisson probs, will use them lots
loc_1_dem_probs = poisson_probs(LOC_1_EXPECTED_DEMAND)
loc_2_dem_probs = poisson_probs(LOC_2_EXPECTED_DEMAND)
loc_1_ret_probs = poisson_probs(LOC_1_EXPECTED_RETURNS)
loc_2_ret_probs = poisson_probs(LOC_2_EXPECTED_RETURNS)

values = dict()
policy = dict()

for state in state_iter():
    values[state] = 0.0 # Init to value 0
    policy[state] = 0 # Init to no transfer
print("Init complete")
while True:
    dMin = 1
    while True:
        delta = 0
        for state in state_iter():
            old_value = values[state]
            best_action = policy[state]
            values[state] = expected_return(state, best_action, values)
            delta = max(delta, abs(values[state] - old_value))
        if delta < dMin:
            break
    policy_stable = True
    for state in state_iter():
        old_action = policy[state]
        policy[state] = greedyAction(state, values)
        if old_action != policy[state]:
            policy_stable = False
        #print("{} : {}".format(state, policy[state]))
    if policy_stable == True:
        break;

Init complete


In [11]:
print_policy(policy)
print(policy)
print(values)

000000011222333344444
000000001112222333333
000000000011112222222
000000000000011111112
100000000000000000011
111000000000000000000
221100000000000000000
322110000000000000000
332211000000000000000
433221000000000000000
443321000000000000000
544321100000000000000
554322100000000000000
554332100000000000000
554432100000000000000
555432100000000000000
555432100000000000000
555432110000000000000
555432210000000000000
555433211100000000000
555443222111000000000
{(0, 0): 0, (0, 1): 0, (0, 2): 0, (0, 3): 0, (0, 4): 0, (0, 5): 0, (0, 6): 0, (0, 7): -1, (0, 8): -1, (0, 9): -2, (0, 10): -2, (0, 11): -2, (0, 12): -3, (0, 13): -3, (0, 14): -3, (0, 15): -3, (0, 16): -4, (0, 17): -4, (0, 18): -4, (0, 19): -4, (0, 20): -4, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (1, 8): -1, (1, 9): -1, (1, 10): -1, (1, 11): -2, (1, 12): -2, (1, 13): -2, (1, 14): -2, (1, 15): -3, (1, 16): -3, (1, 17): -3, (1, 18): -3, (1, 19): -3, (1, 20): -3, (2, 0): 0, (2, 1): 0, (2, 

In [20]:
# Additional problem constraints
# Employee who takes bus can take one car for free from loc1 to loc2
# Any cars above 10 at a location after moves require use of a second lot for a fixed cost of $4.

# Create modified expected return function
def expected_return_mod(state, action, values):
    # Action is given and has known cost
    Vsa = 0
    # Check if the action is 'legal', if not return -inf
    loc_1_avail, loc_2_avail = state
    if action > loc_1_avail or action < -loc_2_avail:
        return -float('inf')
    # Any move has a known cost with prob 1, but positive moves (loc1->loc2) are discounted by 1 due to the one free bus trip.
    Vsa += MOVE_REWARD * abs(action - 1 if action > 0 else action)
    # Update availability based on action; enforce max capacity constraint
    loc_1_avail = min(loc_1_avail - action, MAX_CAPACITY)
    loc_2_avail = min(loc_2_avail + action, MAX_CAPACITY)
    
    for loc_1_dem, loc_1_dem_prob in enumerate(loc_1_dem_probs):
        for loc_2_dem, loc_2_dem_prob in enumerate(loc_2_dem_probs):
            for loc_1_ret, loc_1_ret_prob in enumerate(loc_1_ret_probs):
                for loc_2_ret, loc_2_ret_prob in enumerate(loc_2_ret_probs):
                    # Reward is fulfilled demand * RENTAL_REWARD
                    parking_reward = 0
                    prob = loc_1_dem_prob * loc_2_dem_prob * loc_1_ret_prob * loc_2_ret_prob
                    loc_1_rented = min(loc_1_avail, loc_1_dem)
                    loc_2_rented = min(loc_2_avail, loc_2_dem)
                    rental_reward = (loc_1_rented + loc_2_rented) * RENTAL_REWARD
                    loc_1_avail_final = min(loc_1_avail - loc_1_rented + loc_1_ret, MAX_CAPACITY)
                    loc_2_avail_final = min(loc_2_avail - loc_2_rented + loc_2_ret, MAX_CAPACITY)
                    # If either location has more than 10 vehicles on site overnight the extra parking lot will need to be used at cost 4.
                    if loc_1_avail_final > 10:
                        parking_reward -= 4
                    if loc_2_avail_final > 10:
                        parking_reward -= 4                    
                    next_state_value = values[(loc_1_avail_final, loc_2_avail_final)]
                    Vsa += prob * (rental_reward + parking_reward + next_state_value * DISCOUNT_RATE)
    return Vsa


In [21]:
# Desired output is optimal policy
# Initialize Val and Pol for all s
# s is all possible end-of-day states of vehicle count
# anything from 0 to max_capacity for each branch

# The action for a state is a transfer of vehicles, which results in a new state
# From the new state there's rentals and returns.
# Rentals result in revenue (up to a limit of state availability)
# Returns - Rentals give the end state.
# That end state will have new actions and new probabilities and new states
# ad infinitum... but we don't have to follow all of those, because they're
# encapsulated by the value associated with the new state, which we can just look up.
# The value of the current state is based on the probability weighted values of possible future states
# appropriately discounted.

# Precalc and cache poisson probs, will use them lots
loc_1_dem_probs = poisson_probs(LOC_1_EXPECTED_DEMAND)
loc_2_dem_probs = poisson_probs(LOC_2_EXPECTED_DEMAND)
loc_1_ret_probs = poisson_probs(LOC_1_EXPECTED_RETURNS)
loc_2_ret_probs = poisson_probs(LOC_2_EXPECTED_RETURNS)

values = dict()
policy = dict()

for state in state_iter():
    values[state] = 0.0 # Init to value 0
    policy[state] = 0 # Init to no transfer
print("Init complete")
while True:
    dMin = 1
    while True:
        delta = 0
        for state in state_iter():
            old_value = values[state]
            best_action = policy[state]
            values[state] = expected_return_mod(state, best_action, values)
            delta = max(delta, abs(values[state] - old_value))
        if delta < dMin:
            break
    policy_stable = True
    for state in state_iter():
        old_action = policy[state]
        policy[state] = greedyAction(state, values)
        if old_action != policy[state]:
            policy_stable = False
        #print("{} : {}".format(state, policy[state]))
    if policy_stable == True:
        break;

Init complete


In [22]:
print_policy(policy)
print(policy)
print(values)

000000112223333444445
000000011122223333344
000000000111122222333
000000000000111112222
000000000000000011111
111000000000000000000
221100000000000000000
322100000000000000000
332110000000000000000
432210000000000000000
433211000000000000000
443221000000000000000
543321000000000000000
544321000000000000000
554321000000000000000
554321000000000000000
554321000000000000000
554321100000000000000
554322100000000000000
554332100000000000000
554432100000000000000
{(0, 0): 0, (0, 1): 0, (0, 2): 0, (0, 3): 0, (0, 4): 0, (0, 5): 0, (0, 6): -1, (0, 7): -1, (0, 8): -2, (0, 9): -2, (0, 10): -2, (0, 11): -3, (0, 12): -3, (0, 13): -3, (0, 14): -3, (0, 15): -4, (0, 16): -4, (0, 17): -4, (0, 18): -4, (0, 19): -4, (0, 20): -5, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): -1, (1, 8): -1, (1, 9): -1, (1, 10): -2, (1, 11): -2, (1, 12): -2, (1, 13): -2, (1, 14): -3, (1, 15): -3, (1, 16): -3, (1, 17): -3, (1, 18): -3, (1, 19): -4, (1, 20): -4, (2, 0): 0, (2, 1): 0, (2